In [ ]:
%matplotlib inline

import os
import re
from skimage import io
from particle_model import Particle, Sphere
from records import TERMINAL_VELOCITIES, RADIUS
from plot_helper import *
import json  # import the library that stores the particle configurations

with open('particle_configurations.json') as f:
    configurations = json.load(f)

In [ ]:
# This function creates and initializes particles based on the configuration provided
def create_particle(name):
    # retrieve configuration from the library by name
    configuration = configurations[name]
    p = Particle()  # instantiate a particle object

    # instantiate sphere objects
    for sphere_config in configuration:
        s = Sphere(sphere_config["center"], sphere_config["radius"], sphere_config["material"])
        p.add_sphere(s)

    return p